<a href="https://colab.research.google.com/github/koppalamounika/nndl/blob/main/assignment_8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [27]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.preprocessing import minmax_scale
from sklearn.model_selection import train_test_split
import seaborn as sns
from scipy import stats

In [28]:
# reading data

data = pd.read_excel("/content/drive/MyDrive/pressure_weight.xls")
data.head()

*** No CODEPAGE record, no encoding_override: will use 'ascii'


,X,Y
0,75.1,577.8
1,74.3,577.0
2,88.7,570.9
3,114.6,578.6
4,98.5,572.4


In [29]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30 entries, 0 to 29
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   X       30 non-null     float64
 1   Y       30 non-null     float64
dtypes: float64(2)
memory usage: 608.0 bytes


In [30]:
# missing values

data.isnull()

,X,Y
0,False,False
1,False,False
2,False,False
3,False,False
4,False,False
5,False,False
6,False,False
7,False,False
8,False,False
9,False,False


In [31]:
# outliers

z = np.abs(stats.zscore(data))
print(z)

[[0.90855142 1.61656054]
 [0.95645395 1.59597943]
 [0.09420832 1.43904846]
 [1.45663625 1.63714165]
 [0.49259774 1.47763804]
 [1.30095302 2.6694559 ]
 [1.46861189 0.430574  ]
 [1.68097979 1.25896373]
 [1.00156217 2.78522465]
 [0.01357238 0.20229517]
 [0.98639304 0.47756754]
 [1.52849006 0.25117531]
 [0.54648809 0.22030365]
 [0.16326781 0.12254337]
 [0.98040522 0.28461962]
 [1.4714062  0.22544892]
 [1.44146712 0.22287629]
 [0.74089254 0.12254337]
 [1.16922104 0.16370559]
 [0.9057571  0.10453489]
 [0.10019614 0.24603004]
 [0.04031797 0.05565476]
 [0.98639304 0.31291865]
 [0.66025661 0.56503726]
 [1.77997837 0.10710753]
 [0.86064888 0.05565476]
 [0.84268543 0.04021892]
 [0.50137987 0.12768865]
 [0.35767226 0.27432906]
 [0.46265865 0.18102803]]


In [32]:
f_data = data[(z < 3).all(axis=1)]
f_data.head()

,X,Y
0,75.1,577.8
1,74.3,577.0
2,88.7,570.9
3,114.6,578.6
4,98.5,572.4


In [33]:
print(len(data))
print(len(f_data))
if(len(data) == len(f_data)):
    print('No outliners')

30
30
No outliners


In [34]:
# normalisation

data_n = data.copy()
data_n = (data_n - data_n.min())/(data_n.max() - data_n.min())
data_n.head()

,X,Y
0,0.223183,0.995346
1,0.209343,0.990692
2,0.458478,0.955207
3,0.906574,1.000000
4,0.628028,0.963933


In [35]:
# splitting the data

train_data,test_data = train_test_split(data_n,test_size=0.1)

In [36]:
print(len(train_data))
train_data.head()

27


,X,Y
21,0.474048,0.617219
20,0.456747,0.574171
29,0.619377,0.670739
27,0.340830,0.600931
11,0.927336,0.573008


In [37]:
xa = np.array(train_data['X'])
ya = np.array(train_data['Y'])

In [38]:
# training the data

from sympy import Symbol, Derivative, symbols
m= Symbol('m')
c= Symbol('c')
function = '0.5*(y-m*x-c)**2'
partialderiv= Derivative(function, m)
dfm = partialderiv.doit()
partialderiv= Derivative(function, c)
dfc = partialderiv.doit()
err = []
m1 = -1.0 
c1 = 1.0  
itr = 100  
learning_rate = 0.1  
for i in range(0,itr):
    dfmv1 = 0
    dfcv1 = 0
    er = 0
    for j in range(0,len(xa)):
        er +=  ((ya[j] - (m1 * xa[j]) - c1))**2
        m = symbols('m')
        c = symbols('c')
        x = symbols('x')
        y = symbols('y')
        dfmv = dfm.subs(m, m1) 
        dfmv = dfmv.subs(c, c1) 
        dfmv = dfmv.subs(x, xa[j]) 
        dfmv = dfmv.subs(y, ya[j]) 
        dfmv = round(dfmv,2)
        dfcv = dfc.subs(c, c1) 
        dfcv = dfcv.subs(m, m1) 
        dfcv = dfcv.subs(x, xa[j]) 
        dfcv = dfcv.subs(y, ya[j]) 
        dfcv = round(dfcv,2)
        dfmv1 += dfmv
        dfcv1 += dfcv
    dfmv1 = dfmv1/len(xa)
    dfcv1 = dfcv1/len(xa)
    dfmv1 = round(dfmv1,2)
    dfcv1 = round(dfcv1,2)
    dm = (-1.0)*learning_rate*dfmv1
    dc = (-1.0)*learning_rate*dfcv1 
    m1 = m1 + dm
    m1 = round(m1, 2) 
    c1 = c1 + dc
    c1 = round(c1, 2)
    er = (1/(2*len(train_data['X'])))*(er)
    err.append(er)
print("m = {} , c = {}".format(m1, c1))
print("minimum value obtained at m = {} ,c ={}".format(m1,c1))

m = -0.83 , c = 1.06
minimum value obtained at m = -0.83 ,c =1.06


In [39]:
# error 

train_data_pred =  m1*train_data['X'] + c1

from sklearn.metrics import mean_squared_error
mse = mean_squared_error(train_data['Y'],train_data_pred)
print("mse of training data = {}".format(mse))

mse of training data = 0.08656392798495617


In [40]:
# testing the data

test_data_pred = m1 *test_data['X'] + c1
mse2 = mean_squared_error(test_data['Y'],test_data_pred)
print("mse of testing data = {}".format(mse2))

mse of testing data = 0.062293032285102995


In [41]:
x_min = data['X'].min()
x_max = data['X'].max()
y_min = data['Y'].min()
y_max = data['Y'].max()

In [42]:
# predicting output

x =x1 = float(input("enter pressure of liquid nitrogen value :"))
x = (x - x_min)/(x_max - x_min)
y = m1*x + c1
y = y*(y_max - y_min) + y_min
print("weight of liquid nitrogen based on its pressure {} is : {}".format(x1,y))

enter pressure of liquid nitrogen value :68
weight of liquid nitrogen based on its pressure 68.0 is : 574.605655445772
